In [1]:
import numpy as np
import math

In [4]:
#here we are initalizing the query , key and value vector randomly 
l,d_k,d_v=4,8,8
q=np.random.randn(l,d_k)
k=np.random.randn(l,d_k)
v=np.random.randn(l,d_v)

In [5]:
print("Q\n",q)
print("K\n",k)
print("V\n",v)

Q
 [[ 0.80130725 -0.46483161 -1.98639082 -0.42187247 -0.88257403 -0.15548923
   1.15717148 -0.01476848]
 [-0.20105037  0.41058351 -0.79883495 -0.79533261 -0.53269527 -0.15628997
  -1.14339552 -0.11984383]
 [ 0.71165268 -1.20217112  1.1148881  -0.54266099 -0.3102796  -0.93644262
   0.79381267 -0.56538197]
 [-0.48411164 -0.0961636   0.49070894  0.22509953 -0.51552848 -0.35349724
  -1.91916762 -0.03109942]]
K
 [[ 0.93153299 -0.51414795  0.10662418  1.0737366   0.49185312  0.43111192
  -1.09719383  1.32047072]
 [ 0.80399534  1.69758699  1.82957875  3.1199611   1.79123017  1.10054696
  -0.96653143 -0.00499251]
 [-0.83884894  1.74763009  0.13071176 -0.67575595 -1.47111393  1.1627357
  -0.11741881 -0.00474525]
 [ 0.67117666 -0.91776187  0.89453763  1.62047353  2.07038205  0.89828093
  -0.60132478 -0.39507005]]
V
 [[ 1.19824388  0.68635123 -0.30974763 -0.38080014 -1.10171085  0.0860364
  -1.38004859  2.1638702 ]
 [-0.62967926  0.7662853  -3.01635049 -0.01370378  0.29032149  1.36633909
   0.925

# Self Attention
* here we are applying the self attention formula whihc  is Q.K^T/sqrt(dk)+m)v)V


In [7]:
np.matmul(q,k.T)

array([[-1.46961364, -7.96571422, -0.4773208 , -4.15305079],
       [-0.57064863, -3.42803887,  2.05598895, -3.02353784],
       [-1.35663944, -3.47273369, -2.90838841, -0.03866653],
       [ 1.55116723,  1.59024343,  0.72293845,  0.34850503]])

In [9]:
# why we need sqrt(d_k) in denominator
# this is used to minimize the variance which stabilize the value of  the Q.K^T vector 
q.var(),k.var(),np.matmul(q,k.T).var()

(0.5552160175580109, 1.1698412822885325, 6.5352397977074475)

In [11]:
#in this we are diving the Q.K^T value with sqrt(d_k)
scaled=np.matmul(q,k.T)/math.sqrt(d_k)
q.var(),k.var(),scaled.var()

(0.5552160175580109, 1.1698412822885325, 0.8169049747134309)

# Masking
- it is required in decoder part of the transformer neural network. so that we dont look
- at a feature word when trying to generate the curent content context word.
- This is to ensure words don't get content form word generated in the future
- Not required in the encoder, but required in the decoder

In [13]:
mask=np.tril(np.ones((l,l)))
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [14]:
mask[mask==0]=-np.infty
mask[mask==1]=0

In [15]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [16]:
scaled+mask

array([[-0.51958688,        -inf,        -inf,        -inf],
       [-0.20175476, -1.21199477,        -inf,        -inf],
       [-0.47964447, -1.22779677, -1.02827058,        -inf],
       [ 0.54842043,  0.56223596,  0.25559734,  0.12321514]])

# now we need to pass these into ssoftmax activation

In [17]:
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

In [20]:
attention=softmax(scaled+mask) # we are adding mask becuse i want to focus on recent word only
#it means it doesn't focus on next word
#here 1 represent let say my is focus on my and in next line 0.733 and 0.266 is focusimg on my name etc

In [21]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.73306712, 0.26693288, 0.        , 0.        ],
       [0.48757104, 0.23073819, 0.28169077, 0.        ],
       [0.29293719, 0.29701235, 0.21857629, 0.19147417]])

In [24]:
#new set of matrix that better encapluate with our context word
new_v=np.matmul(attention,v)
new_v

array([[ 1.19824388,  0.68635123, -0.30974763, -0.38080014, -1.10171085,
         0.0860364 , -1.38004859,  2.1638702 ],
       [ 0.71031109,  0.70768826, -1.03222894, -0.28281005, -0.73013165,
         0.42779129, -0.76465256,  1.72814156],
       [ 0.40254755,  0.83499895, -0.94729156, -0.15661906, -0.58350672,
         0.31451642, -0.46137248,  1.1290593 ],
       [ 0.07179775,  0.66263767, -1.05417465, -0.39890449, -0.63373607,
         0.36931117, -0.02233477,  0.99883798]])

In [25]:
v

array([[ 1.19824388,  0.68635123, -0.30974763, -0.38080014, -1.10171085,
         0.0860364 , -1.38004859,  2.1638702 ],
       [-0.62967926,  0.7662853 , -3.01635049, -0.01370378,  0.29032149,
         1.36633909,  0.92538498,  0.5315174 ],
       [-0.12918569,  1.14857396, -0.35599435,  0.11434539, -0.40233007,
        -0.15158129, -0.00717955, -0.17260952],
       [-0.33400323, -0.08913666,  0.05362202, -1.61001858, -1.61532621,
        -0.1492609 ,  0.56744617,  1.27861138]])

In [28]:
#here are all the logics are defined which we practise seprately above
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

#here mask is option becuase the mask in only used in decoder part not in encoder part
# so if we are usin this funnction into encoder the we don't pass the mask value and else pass the mask value if we are going to use this funtion insdie the decoder
def scaled_dot_product_attention(q,k,v,mask=None):
    d_k=q.shape[-1]
    scaled=np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled=scaled+mask
    attention=softmax(scaled)
    out=np.matmul(attention,v)
    return out,attention

In [29]:
values,attention=scaled_dot_product_attention(q,k,v,mask=None)
print("Q\n",q)
print("K\n",k)
print("V\n",v)
print("New V\n",values)
print("Attention\n",attention)

Q
 [[ 0.80130725 -0.46483161 -1.98639082 -0.42187247 -0.88257403 -0.15548923
   1.15717148 -0.01476848]
 [-0.20105037  0.41058351 -0.79883495 -0.79533261 -0.53269527 -0.15628997
  -1.14339552 -0.11984383]
 [ 0.71165268 -1.20217112  1.1148881  -0.54266099 -0.3102796  -0.93644262
   0.79381267 -0.56538197]
 [-0.48411164 -0.0961636   0.49070894  0.22509953 -0.51552848 -0.35349724
  -1.91916762 -0.03109942]]
K
 [[ 0.93153299 -0.51414795  0.10662418  1.0737366   0.49185312  0.43111192
  -1.09719383  1.32047072]
 [ 0.80399534  1.69758699  1.82957875  3.1199611   1.79123017  1.10054696
  -0.96653143 -0.00499251]
 [-0.83884894  1.74763009  0.13071176 -0.67575595 -1.47111393  1.1627357
  -0.11741881 -0.00474525]
 [ 0.67117666 -0.91776187  0.89453763  1.62047353  2.07038205  0.89828093
  -0.60132478 -0.39507005]]
V
 [[ 1.19824388  0.68635123 -0.30974763 -0.38080014 -1.10171085  0.0860364
  -1.38004859  2.1638702 ]
 [-0.62967926  0.7662853  -3.01635049 -0.01370378  0.29032149  1.36633909
   0.925

In [30]:
#using these function as decoder part so we need to pass the mask value
def softmax(x):
    return (np.exp(x).T/np.sum(np.exp(x),axis=-1)).T

#here mask is option becuase the mask in only used in decoder part not in encoder part
# so if we are usin this funnction into encoder the we don't pass the mask value and else pass the mask value if we are going to use this funtion insdie the decoder
def scaled_dot_product_attention(q,k,v,mask=None):
    d_k=q.shape[-1]
    scaled=np.matmul(q,k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled=scaled+mask
    attention=softmax(scaled)
    out=np.matmul(attention,v)
    return out,attention

In [33]:
#passing the mask value becuase we use this function as decoder
values,attention=scaled_dot_product_attention(q,k,v,mask=mask)
print("Q\n",q)
print("K\n",k)
print("V\n",v)
print("New V\n",values)
print("Attention\n",attention)

Q
 [[ 0.80130725 -0.46483161 -1.98639082 -0.42187247 -0.88257403 -0.15548923
   1.15717148 -0.01476848]
 [-0.20105037  0.41058351 -0.79883495 -0.79533261 -0.53269527 -0.15628997
  -1.14339552 -0.11984383]
 [ 0.71165268 -1.20217112  1.1148881  -0.54266099 -0.3102796  -0.93644262
   0.79381267 -0.56538197]
 [-0.48411164 -0.0961636   0.49070894  0.22509953 -0.51552848 -0.35349724
  -1.91916762 -0.03109942]]
K
 [[ 0.93153299 -0.51414795  0.10662418  1.0737366   0.49185312  0.43111192
  -1.09719383  1.32047072]
 [ 0.80399534  1.69758699  1.82957875  3.1199611   1.79123017  1.10054696
  -0.96653143 -0.00499251]
 [-0.83884894  1.74763009  0.13071176 -0.67575595 -1.47111393  1.1627357
  -0.11741881 -0.00474525]
 [ 0.67117666 -0.91776187  0.89453763  1.62047353  2.07038205  0.89828093
  -0.60132478 -0.39507005]]
V
 [[ 1.19824388  0.68635123 -0.30974763 -0.38080014 -1.10171085  0.0860364
  -1.38004859  2.1638702 ]
 [-0.62967926  0.7662853  -3.01635049 -0.01370378  0.29032149  1.36633909
   0.925